In [3]:
!pip install ibmos2spark

### Please upload hmp.parquet to object storage before running the next cell.

In [36]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-5119afa5-9f36-4561-8de3-537f498c07d2',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'UdCSHHwYJp9QLwhI_Gt3F7rxwpCcfZ48WNZKvwQlD1Hk'
}

configuration_name = 'os_ab26667fc49e46aaa28ad8a08f0a62c0_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read.parquet(cos.url('hmp.parquet', 'scalablemachinelearningonbigdatau-donotdelete-pr-zvgztoifq8gdmt'))
df_data_1.show()


+---+---+---+--------------------+-----------+
|  x|  y|  z|              source|      class|
+---+---+---+--------------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|
| 21| 51| 33|Accelerometer-201...|Brush_teeth|
| 20| 50| 34|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 21| 49| 33|Accelerometer-201...|Brush_teeth|
| 20| 51| 35|Accelerometer-201...|Brush_teeth|
| 18| 49| 34|Accelerometer-201...|Brush_teeth|
| 19| 48| 34|Accelerometer-201...|Brush_teeth|
| 16| 53| 34|Accelerometer-201...|Brush_teeth|
| 18| 52| 35|

In [37]:
df = df_data_1

In [38]:
# creating temporary query view
df.createOrReplaceTempView('df')

In [39]:
# feature engineering one additional column
df_energy = spark.sql('''

select sqrt(sum(x*x)+sum(y*y)+sum(z*z)) as label, class from df group by class

'''
)

df_energy.createOrReplaceTempView('df_energy')

In [40]:
df_join = spark.sql('''

select * from df inner join df_energy on df.class = df_energy.class

''')
df_join.show()

+---+---+---+--------------------+-----------+-----------------+-----------+
|  x|  y|  z|              source|      class|            label|      class|
+---+---+---+--------------------+-----------+-----------------+-----------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 20| 50| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 52| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 50| 34|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|
| 22| 51| 35|Accelerometer-201...|Brush_teeth|11785.39634462923|Brush_teeth|

In [41]:
# vectorAssembler and normalizer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer

vectorAssembler = VectorAssembler(inputCols = ['x', 'y', 'z'],
                                  outputCol = 'features')
normalizer = Normalizer(inputCol = 'features',
                       outputCol = 'features_norm', p = 1.0)

In [42]:
# Import linearRegression
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(maxIter = 10, regParam = 0.3, elasticNetParam = 0.8)

In [43]:
# Pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages = [vectorAssembler, normalizer, lr])

In [45]:
# modeling
model = pipeline.fit(df_join)

In [48]:
# predict
prediction = model.transform(df_join)

In [51]:
# finding r2 score of the linear regression which is stage number 3 in stages so index = 2
model.stages[2].summary.r2

0.03259100556263628